In [1]:
# ------------>>>>>>>> RUN THIS CODE CELL <<<<<<<<------------
# === CELL TYPE: IMPORTS AND SETUP 

import time      # for testing use only
import os         # for testing use only

import requests
import re
import bs4
from bs4 import BeautifulSoup, Comment  
import pandas as pd
import scipy as sc
import numpy as np
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager

In [2]:
driver = webdriver.Edge(r'C:\Users\ItanM\Documents\Python Scripts\edgedriver_win64\msedgedriver.exe')

C:\Users\ItanM\AppData\Local\Temp/ipykernel_620/3669523527.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(r'C:\Users\ItanM\Documents\Python Scripts\edgedriver_win64\msedgedriver.exe')


In [3]:
def load_soup_obj(url):
    path = requests.get(url)
    soup_path = path.text
    soup = BeautifulSoup(soup_path, "html.parser")
    return soup

In [27]:
def get_page_values(soup):
    type_list = soup.find_all("div", attrs ={"class":"type"})
    temp_value_list = soup.find_all("div", attrs ={"class":"value"})
    key_list = []
    value_list=[]
    for key in type_list:
        insert_key = key.get_text(strip=True)
        if(insert_key == 'Season'):
            continue
        key_list.append(insert_key)
    for val in temp_value_list:
        insert_value = val.get_text(strip=True)
        value_list.append(insert_value)
    dic = dict(zip(key_list, value_list))
    return dic

In [40]:
def scroll_screen(driver):
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        #if (screen_height) * i > scroll_height:
        if(i > 10):
            break 


In [47]:
def create_data_frame(broken_url_list):
    index = 0
    fixed_url_list = []
    df = pd.DataFrame()
    for url_list in broken_url_list:
        for url in url_list:
            new_url = base_URL + url['href'] 
            temp_soup = load_soup_obj(new_url)
            url_dic = get_page_values(temp_soup)
            df = df.append(url_dic, ignore_index=True)
    return df

In [15]:
base_URL = 'https://anilist.co'
driver.get('https://anilist.co/search/anime/top-100')
time.sleep(2)

In [41]:
broken_url_list = []
for i in range(0,5):
    main_soup = BeautifulSoup(driver.page_source, "html.parser")
    broken_url_list.append(main_soup.find_all("a", attrs = {"class" : "cover"}))
    scroll_screen(driver)
broken_url_list

[[<a class="cover" data-v-4d8937d6="" href="/anime/114129/Gintama-THE-FINAL/"><!-- --> <img class="image loaded" data-v-4d8937d6="" src="https://s4.anilist.co/file/anilistcdn/media/anime/cover/large/bx114129-TI4mHjFNI0E0.png"/> <!-- --></a>,
  <a class="cover" data-v-4d8937d6="" href="/anime/124194/Fruits-Basket-The-Final/"><!-- --> <img class="image loaded" data-v-4d8937d6="" src="https://s4.anilist.co/file/anilistcdn/media/anime/cover/large/bx124194-r9MueM46SgLV.jpg"/> <!-- --></a>,
  <a class="cover" data-v-4d8937d6="" href="/anime/20996/Gintama/"><!-- --> <img class="image loaded" data-v-4d8937d6="" src="https://s4.anilist.co/file/anilistcdn/media/anime/cover/large/bx20996-qYCvgsFuF9wk.png"/> <!-- --></a>,
  <a class="cover" data-v-4d8937d6="" href="/anime/104578/Shingeki-no-Kyojin-3-Part-2/"><!-- --> <img class="image loaded" data-v-4d8937d6="" src="https://s4.anilist.co/file/anilistcdn/media/anime/cover/large/bx104578-LaZYFkmhinfB.jpg"/> <!-- --></a>,
  <a class="cover" data-v-4d

In [28]:
url ='https://anilist.co/anime/124194/Fruits-Basket-The-Final/'
soup = load_soup_obj(url)
dic = get_page_values(soup)
dic

{'Format': 'TV',
 'Episodes': '13',
 'EpisodeDuration': '24 mins',
 'Status': 'Finished',
 'Average Score': '91%',
 'Mean Score': '91%',
 'Popularity': '76796',
 'Favorites': '6002',
 'Studios': 'TMS Entertainment',
 'Producers': 'TMS MusicFunimation',
 'Source': 'Manga',
 'Hashtag': '#フルーツバスケット #フルバ #fruba #furuba',
 'Genres': 'ComedyDramaPsychologicalRomanceSlice of LifeSupernatural',
 'Romaji': 'Fruits Basket: The Final',
 'English': 'Fruits Basket The Final Season',
 'Native': 'フルーツバスケットThe Final',
 'Synonyms': 'FurubaFrubaフルバFruits Basket Season 3水果篮子 最终季'}

In [48]:
df = create_data_frame(broken_url_list)


KeyboardInterrupt: 

In [46]:
df

,Format,Episodes,Duration,Status,Average Score,Mean Score,Popularity,Favorites,Studios,Source,Hashtag,Genres,Romaji,English,Native,EpisodeDuration,Producers,Synonyms
0,Movie,1,"1 hour, 44 mins",Finished,92%,93%,18572,1507,Bandai Namco Pictures,Manga,#銀魂ザファイナル,ActionComedyDramaSci-Fi,Gintama: THE FINAL,Gintama: THE VERY FINAL,銀魂 THE FINAL,NaN,NaN,NaN
1,TV,13,NaN,Finished,91%,91%,76908,6002,TMS Entertainment,Manga,#フルーツバスケット #フルバ #fruba #furuba,ComedyDramaPsychologicalRomanceSlice of LifeSu...,Fruits Basket: The Final,Fruits Basket The Final Season,フルーツバスケットThe Final,24 mins,TMS MusicFunimation,FurubaFrubaフルバFruits Basket Season 3水果篮子 最终季
2,TV,51,NaN,Finished,90%,90%,67417,4004,Bandai Namco Pictures,Manga,#gintama,ActionComedyDramaSci-Fi,Gintama°,Gintama Season 4,銀魂°,24 mins,Aniplex,NaN
3,TV,10,NaN,Finished,90%,90%,296196,20919,Wit Studio,Manga,#shingeki,ActionDramaFantasyMystery,Shingeki no Kyojin 3 Part 2,Attack on Titan Season 3 Part 2,進撃の巨人３ Part.2,24 mins,Pony CanyonKodanshaProduction I.GDentsuPony Ca...,SnK 3AoT 3Shingeki no Kyojin Season 3 (2019)L'...
4,TV,64,NaN,Finished,90%,90%,377415,40816,bones,Manga,NaN,ActionAdventureDramaFantasy,Hagane no Renkinjutsushi: FULLMETAL ALCHEMIST,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,25 mins,AniplexSquare EnixMainichi Broadcasting System...,FMAFMABHagarenالخيميائي الفولاذيאלכימאי המתכת:...
